In [ ]:
%matplotlib inline
import numpy as np
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
from os import listdir
from os.path import join
from sklearn import metrics

# Data generation 

Arguments:

-mu: mean value of the noise;

-sigma: standard deviation;

Function: $sin(2\pi x)+N(0,0.3)$

Outputs:

-x: np.array of size (1000,)




In [ ]:
def generate_data(mu=0,sigma=0.3):
    t=np.arange(0,10,0.01)
    s = np.random.normal(mu, sigma, 1000)
    x=(np.sin(2*np.pi*t)+s)
    return x

# Data preparation

Arguments: 

-time_wind: length of the sequence of the historical data;

-output: how many steps ahead prediction is going to be performed;

-nb_var: number of variables of the data set;

Outputs:

-x_batches: input for the training set. Size: (899, 2, 1);

-y_batches: output for the training set. Size: (899, 1);

-x_test: inputs for the test set; Size: (96, 2, 1);

-y_test: output for the test set; Size: (96, 1);

! Note, the size of the first dimension will vary depending on the size of the time_wind. The second dimension refers to the time_wind size, thisrd dimension: number of variables;


In [ ]:
def data_preparation(data,time_wind=2,output=1,nb_var=1,batch_size=1):
    data_train=data[:901]
    x_train=[]
    for i in range((len(data_train)-1)-(time_wind-1)):
        x_train.append(data_train[i:(i+time_wind)])
    x_train=np.array(x_train)
    x_batches=(x_train[:(len(x_train)-(len(x_train)%batch_size))]).reshape(-1,time_wind,nb_var)
    y_train=data_train[time_wind:len(data_train)]
    y_batches=(y_train[:(len(y_train)-(len(y_train)%batch_size))]).reshape(-1,output)
    data_test=data[902:]
    x_t=[]
    for i in range((len(data_test)-1)-(time_wind-1)):
        x_t.append(data_test[i:(i+time_wind)])
    x_t=np.array(x_t)
    x_test=(x_t[:(len(x_t)-(len(x_t)%batch_size))]).reshape(-1,time_wind,nb_var)
    y_t=data_test[time_wind:len(data_test)]
    y_test=(y_t[:(len(y_t)-(len(y_t)%batch_size))]).reshape(-1,output)
    return x_batches, y_batches, x_test, y_test

# TensorFlow computation graph

Arguments:

-epochs: how many times the network is going to be trained;

-time_wind: the length of the historical sequence;

-hidden: number of hidden layers;

-nb_var: amount of input variables;

-output: how many steps ahead prediction is going to be performed;

-lr: learning rate.

Outputs:

-mse_test: the mean squared error for the final prediction vector;

-y_pred: the vector of predictions.


In [ ]:
def run_graph(epochs,time_wind,hidden=10,nb_var=1,output=1,lr=0.01):

    from tensorflow.contrib import rnn
    tf.reset_default_graph()
    X=tf.placeholder(tf.float32,[None,time_wind,nb_var])
    Y=tf.placeholder(tf.float32,[None,output])
 
    tf.set_random_seed(1)
    x=tf.unstack(X,time_wind,1) # we use static RNN, so the dimension of the input should be reduced. 
    
    #After unstack dimension: we receive "time_wind" tensors of the shape (length, nb_var)
    
    basic_cell=tf.contrib.rnn.BasicRNNCell(num_units=hidden) # here the first set of weights and biases is defined
    rnn_output, states=rnn.static_rnn(basic_cell,x,dtype=tf.float32)

    stacked_rnn_output=tf.reshape(rnn_output[-1],[-1,hidden])
    stacked_outputs=tf.layers.dense(stacked_rnn_output,output) # the output layer. (+additional set of weights and biases)
    loss=tf.reduce_mean(tf.squared_difference(stacked_outputs, Y))
    optimizer=tf.train.AdamOptimizer(learning_rate=lr)
    training_op=optimizer.minimize(loss)
    init=tf.global_variables_initializer()
    with tf.Session() as sess:
        init.run()
        for ep in range(epochs):
            sess.run(training_op,feed_dict={X: x_batches, Y: y_batches})
            if ep % 100 == 0:
                mse=loss.eval(feed_dict={X: x_batches, Y: y_batches})
                print(ep,"\tMSE:",mse)
        y_pred=sess.run(stacked_outputs,feed_dict={X:x_test})
        mse_test=loss.eval(feed_dict={X:x_test,Y:y_test})
        print(mse)
        print(mse_test)
    sess.close() 
    return mse_test, y_pred

# Visualisastion

In [ ]:
def visualisation():
    plt.plot(y_test,label='Actual')
    plt.plot(y_pred,label='Forecast')
    plt.legend(loc="upper left")
    return plt.show

# Experiments

## Naive models

### Average naive model

In [ ]:
mse=[]
for i in range (10):
    np.random.seed(i)
    x_batches, y_batches, x_test, y_test=data_preparation(generate_data(),1)
    mse.append(metrics.mean_squared_error(x_batches[:,0],np.full(len(x_batches),np.mean(x_batches))))
print("Mean of MSE for the naive model:{}".format(np.mean(mse)))
print("Variance of MSE for the naive model:{}".format(np.var(mse)))

### Constant naive model

In [ ]:
mse=[]
for i in range (10):
    np.random.seed(i)
    x_batches, y_batches, x_test, y_test=data_preparation(generate_data(),1)
    x_t1=y_test[:96]
    x_t=y_test[1:]
    mse.append(metrics.mean_squared_error(x_t,x_t1))
print("Mean of MSE for the naive model:{}".format(np.mean(mse)))
print("Variance of MSE for the naive model:{}".format(np.var(mse)))

# Time_wind
The aim of the first  experiment is to define the optimal size of the time_wind

In [ ]:
mse_mean=[]
mse_var=[]
for j in range(1,9,1):
    mse=[]
    for i in range (10):
        np.random.seed(i)
        x_batches, y_batches, x_test, y_test=data_preparation(generate_data(),j)
        mse_test,y_pred=run_graph(1000,j,hidden=1,lr=0.1)
        mse.append(mse_test)
    mse_mean.append(np.mean(mse))
    mse_var.append(np.var(mse))
    print("Mean of MSE for the model with time_wind={} equals{}".format(j, np.mean(mse)))
    print("Variance of MSE for the model with time_wind={} equals{}".format(j, np.var(mse)))
visualisation()

In [ ]:
plt.plot(mse_mean)

Summary: the highest accuracy was obtained with time_wind=6. Then MSE=0.10424. The accuracy of the basic model with time_wind=1 is equal to 0.133629, what is better the the results of the naive model.    

# Hidden layers
The aim of the second experiment is to discover wether the increment of the number of hidden layers will improve the accuracy

In [ ]:
mse_mean=[]
mse_var=[]
for j in range(1,11,1):
    mse=[]
    for i in range (10):
        np.random.seed(i)
        x_batches, y_batches, x_test, y_test=data_preparation(generate_data(),6)
        mse_test,y_pred=run_graph(1000,6,hidden=j,lr=0.1)
        mse.append(mse_test)
    mse_mean.append(np.mean(mse))
    mse_var.append(np.var(mse))
    print("Mean of MSE for the model with hidden={} equals{}".format(j, np.mean(mse)))
    print("Variance of MSE for the model with hidden={} equals{}".format(j, np.var(mse)))
visualisation()

In [ ]:
plt.plot(mse_mean)

Summary: additional hidden layers make the quality of predictions worse

# Optimisation of the training time

# Learning rate: 0.1

In [ ]:
mse_mean=[]
mse_var=[]
for j in range(1000,11000,1000):
    mse=[]
    for i in range (10):
        np.random.seed(i)
        x_batches, y_batches, x_test, y_test=data_preparation(generate_data(),6)
        mse_test,y_pred=run_graph(j,6,hidden=1,lr=0.1)
        mse.append(mse_test)
    mse_mean.append(np.mean(mse))
    mse_var.append(np.var(mse))
    print("Mean of MSE for the model with epochs={} equals {}".format(j, np.mean(mse)))
    print("Variance of MSE for the model with epochs={} equals {}".format(j, np.var(mse)))
visualisation()

In [ ]:
plt.plot(mse_mean)

Summary: With the learning rate 0.1 the lowest MSE is obtained with number of training epochs=5000. MSE=0.104167

# Learning rate: 0.01

In [ ]:
mse_mean=[]
mse_var=[]
for j in range(1000,11000,1000):
    mse=[]
    for i in range (10):
        np.random.seed(i)
        x_batches, y_batches, x_test, y_test=data_preparation(generate_data(),6)
        mse_test,y_pred=run_graph(j,6,hidden=1,lr=0.01)
        mse.append(mse_test)
    mse_mean.append(np.mean(mse))
    mse_var.append(np.var(mse))
    print("Mean of MSE for the model with epochs={} equals {}".format(j, np.mean(mse)))
    print("Variance of MSE for the model with epochs={} equals {}".format(j, np.var(mse)))
visualisation()

In [ ]:
plt.plot(mse_mean)

In [ ]:
plt.plot(mse_mean[2:])

Summary: With the learning rate of 0.01, the highest ouput is obtained with number of epochs=5000. MSE=0.10420541

# Learning rate: 0.001
As the learning rate is decreased, it makes sense to start the experiment with epochs=3000, and finishes with epochs=16000

In [ ]:
mse_mean=[]
mse_var=[]
for j in range(3000,17000,1000):
    mse=[]
    for i in range (10):
        np.random.seed(i)
        x_batches, y_batches, x_test, y_test=data_preparation(generate_data(),6)
        mse_test,y_pred=run_graph(j,6,hidden=1,lr=0.001)
        mse.append(mse_test)
    mse_mean.append(np.mean(mse))
    mse_var.append(np.var(mse))
    print("Mean of MSE for the model with epochs={} equals {}".format(j, np.mean(mse)))
    print("Variance of MSE for the model with epochs={} equals {}".format(j, np.var(mse)))
visualisation()

In [ ]:
plt.plot(mse_mean[5:])

Summary: with the lr=0.001, the highest accuracy was obtained with the epochs=12000. MSE=0.10420661